In [246]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

from torch.utils import data
import pandas as pd
import tqdm
import sys

In [247]:
import numpy as np
from scipy.io import loadmat
from scipy.signal import resample

In [248]:
device = torch.device("cuda:0")

In [249]:
data_points = os.listdir("./Training_WFDB/")

In [250]:
data_points = set([i[:-4] for i in data_points])

In [251]:
data_points = list(data_points)

In [252]:
label_types = {
    "AF" : 0,
    "I-AVB" : 1,
    "LBBB" : 2,
    "Normal" : 3,
    "PAC" : 4,
    "PVC" : 5,
    "RBBB" : 6,
    "STD" : 7,
    "STE" : 8
} 

In [253]:
labels = []
for i in data_points:
    with open("./Training_WFDB/" + i + ".hea", "r") as f:
        file = f.readlines()
    for line in file:
        if line.startswith("#Dx"):
            label = line.split(":")[1].strip()
            labels.append(label)

In [254]:
X_train, X_test, y_train, y_test = train_test_split(data_points, labels, test_size=0.2, shuffle=True)

In [255]:
y_test_labels = []
y_train_labels = []
for i in y_test:
    y_test_labels.append(label_types[i.split(",")[0]])
    
for i in y_train:
    y_train_labels.append(label_types[i.split(",")[0]])

y_test_labels = np.array(y_test_labels)
y_train_labels = np.array(y_train_labels)

In [256]:
class DataSet(data.Dataset):
    def __init__(self, input_x, output_y):
        self.inputs = input_x
        self.outputs = output_y
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        file = self.inputs[idx]
        data = loadmat("./Training_WFDB/" + file + ".mat")["val"]
        data_prime = []
        for index, sig in enumerate(data):
            data_prime.append(resample(sig, int(len(sig)/10)))
        data_prime = np.array(data_prime)
        data_prime = np.pad(data_prime, ((0, 0), (0, 7200 - data_prime.shape[1])), 'constant', constant_values=(0))
        return torch.Tensor(data_prime.T).to(device), self.outputs[idx]

In [257]:
dataset_train = DataSet(X_train, y_train)
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True)

dataset_test = DataSet(X_test, y_test)
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=True)

In [258]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(12, 1)
        self.convLayer1 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=5)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.convlayer2 = nn.Conv1d(in_channels=3, out_channels=5, kernel_size=5)
        self.maxpool2 = nn.MaxPool1d(kernel_size=5)
        self.layer2 = nn.Linear(5 * 287, 9)
        
        
    def forward(self, inp):
        inp = torch.relu(self.layer1(inp))
        inp = inp.transpose(1, 2)
        inp = torch.relu(self.convLayer1(inp))        
        inp = self.maxpool1(inp)        
        inp = torch.relu(self.convlayer2(inp))
        inp = self.maxpool2(inp)
        inp = inp.view(-1, 5 * inp.shape[2])
        inp = self.layer2(inp)
        
        return inp

In [259]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.convlayer1 = nn.Conv2d(1, 3, kernel_size=(5, 12))
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.convlayer2 = nn.Conv1d(3, 5, kernel_size=5)
        self.convlayer3 = nn.Conv1d(5, 7, kernel_size=5)
        
        self.layer1 = nn.Linear(7 * 56, 9)
        
    def forward(self, inp):
        inp = torch.relu(self.convlayer1(inp.unsqueeze(1)).squeeze(3))
        inp = self.maxpool1(inp)
        inp = torch.relu(self.convlayer2(inp))
        inp = self.maxpool1(inp)
        inp = torch.relu(self.convlayer3(inp))
        inp = self.maxpool1(inp)
        inp = inp.view(-1, inp.shape[1] * inp.shape[2])
        inp = self.layer1(inp)
        
        return inp

In [260]:
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.layer1 = nn.Linear(12, 1)
        self.layer2 = nn.Linear(72000, 9)
    
    def forward(self, inp):
        inp = self.layer1(inp)
        inp = torch.sigmoid(inp)
        inp = inp.view(-1, 72000)
        inp = self.layer2(inp)
        
        return inp

In [261]:
net = Model2().to(device)

In [262]:
# net = Model().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=5)
num_epochs = 100

In [263]:
def train():
    epoch_loss = []
    points = 0
    predictions = []
    for inp, outp in dataloader_train:
        predicted = net(inp)
        predictions.extend(torch.argmax(predicted, dim=1).tolist())
        outp = list(outp)
        points += 32
        print(points, end='\r')
        for idx, out in enumerate(outp):  
            outp[idx] = label_types[out.split(",")[0]]
        outp = torch.Tensor(outp).long().to(device)
        loss = criterion(predicted, outp)
        epoch_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    return np.mean(epoch_loss), predictions

In [264]:
def validation():
    loss = []
    predictions = []
    for inp, outp in dataloader_test:
        prediction = net(inp)
        predictions.extend(torch.argmax(prediction, dim=1).tolist())
        outp = list(outp)
#         print(outp)
        for idx, out in enumerate(outp):  
            outp[idx] = label_types[out.split(",")[0]]
        outp = torch.Tensor(outp).long().to(device)
        loss.append(criterion(prediction, outp).item())
    return np.mean(loss), predictions

In [265]:
val_loss_store = 1e10
for epoch in range(num_epochs):
    training_loss, train_predictions = train()
    validation_loss, val_predictions = validation()
    val_predictions = np.array(val_predictions)
    train_predictions = np.array(train_predictions)
    print((train_predictions == 6).sum())
    val_accuracy = (val_predictions == y_test_labels).sum() / len(val_predictions)
    train_accuracy = (train_predictions == y_train_labels).sum() / len(train_predictions)
    scheduler.step(validation_loss)
    if validation_loss < val_loss_store:
        torch.save(net.state_dict(), "model.tar")
    print("Epoch Number : {0}/{1} Training Loss : {2} Validation Loss : {3} Train Accuracy : {4} Validation Accuracy : {5}".format(epoch, num_epochs, training_loss, validation_loss, train_accuracy, val_accuracy))
    sys.stdout.flush()

4075
Epoch Number : 0/100 Training Loss : 2.8138948803724246 Validation Loss : 2.064450879429662 Train Accuracy : 0.2103253953826577 Validation Accuracy : 0.24273255813953487


KeyboardInterrupt: 